## I. Data Ingestion
#### This marks the first step towards the development of our project

#### 1. Import the required libraries 

In [1]:
import os # file and path handling
import glob # get list of all CSV files present in the folder
import numpy as np # for mathematical computations and vectorized summarizations
import pandas as pd # data cleaning
from sklearn.preprocessing import LabelEncoder, StandardScaler # for encoding categorical columns
from sklearn.utils.class_weight import compute_class_weight # balanced weighting
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV # training and testing split, fine-tuning techniques
from sklearn.metrics import accuracy_score, classification_report # performance of the model
from sklearn.ensemble import RandomForestRegressor # for regression
from sklearn.metrics import mean_absolute_error, mean_squared_error # for error checking
import xgboost as xgb # for classification
import requests # using the github rest api for data retrieval
import pickle # for importing pickle files

#### 2. Extract Datasets 

In [2]:
directory = r'C:\PROJECT\data\raw-data\match-data-1' 
files = glob.glob(os.path.join(directory, 'pl*.csv')) # extract all the csv files present inside the folder 
files = sorted(files, key=lambda x: int(os.path.basename(x).replace('pl', '').replace('.csv', ''))) # sorting the files by 'pl' name
good_df = [] # list to maintain all the error-free dataframes
bad_files = [] # list to maintain all the bad files

In [3]:
# detect errors while reading the csv files one by one
def checkError(files):
    for file in files:
        try:
            temp_df = pd.read_csv(file)
            good_df.append(temp_df)
            print(f"Successfully read file: {file}")
        except Exception as e:
            bad_files.append(file)
            print(f"Error occured: {e}, in file: {file}")

In [4]:
checkError(files)

Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl0.csv
Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl1.csv
Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl2.csv
Error occured: Error tokenizing data. C error: Expected 57 fields in line 305, saw 72
, in file: C:\PROJECT\data\raw-data\match-data-1\pl3.csv
Error occured: 'utf-8' codec can't decode byte 0xa0 in position 75614: invalid start byte, in file: C:\PROJECT\data\raw-data\match-data-1\pl4.csv
Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl5.csv
Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl6.csv
Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl7.csv
Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl8.csv
Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl9.csv
Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl10.csv
Successfully read file: C:\PROJECT\data\raw-data\match-data-1\pl11.csv
Successfu

In [5]:
# removing errors in 2003-04 file
file1 = r"..\Datasets\pl3.csv"
pl3 = pd.read_csv(file1, sep=",", engine='python', header=0, on_bad_lines="skip")
good_df.insert(3, pl3)

In [6]:
# removing errors in 2004-05 file 
file2 = r"..\Datasets\pl4.csv"
pl4 = pd.read_csv(file2, sep=",", engine='python', header=0, on_bad_lines="skip", encoding="latin1")
good_df.insert(4, pl4)

#### 3. Merging the Datasets

In [7]:
# analyzing the difference in shapes of all datasets
df_list = good_df.copy()
start = 2000
for temp_df in df_list:
    print(f"{start} PL dataset's shape = {temp_df.shape}")
    start += 1

2000 PL dataset's shape = (380, 45)
2001 PL dataset's shape = (380, 48)
2002 PL dataset's shape = (380, 53)
2003 PL dataset's shape = (335, 57)
2004 PL dataset's shape = (335, 57)
2005 PL dataset's shape = (380, 68)
2006 PL dataset's shape = (380, 68)
2007 PL dataset's shape = (380, 71)
2008 PL dataset's shape = (380, 71)
2009 PL dataset's shape = (380, 71)
2010 PL dataset's shape = (380, 71)
2011 PL dataset's shape = (380, 71)
2012 PL dataset's shape = (380, 74)
2013 PL dataset's shape = (380, 68)
2014 PL dataset's shape = (381, 68)
2015 PL dataset's shape = (380, 65)
2016 PL dataset's shape = (380, 65)
2017 PL dataset's shape = (380, 65)
2018 PL dataset's shape = (380, 62)
2019 PL dataset's shape = (380, 106)
2020 PL dataset's shape = (380, 106)
2021 PL dataset's shape = (380, 106)
2022 PL dataset's shape = (380, 106)
2023 PL dataset's shape = (380, 106)
2024 PL dataset's shape = (380, 120)
2025 PL dataset's shape = (60, 132)


In [8]:
original_df = pd.concat(good_df, ignore_index=True)
print(f"Original shape after merging raw datasets = {original_df.shape}")

Original shape after merging raw datasets = (9471, 219)


## II. Data Structuring
#### This step comprises of techniques to refine the structure of the dataset and reduce complexity 

In [9]:
# preparing a list of columns that need to be removed to reduce redundancy and noise from the data set
removable = ['Div', 'AHCh', 'HHW', 'AHW', 'HO', 'AO',
             'IWH', 'IWD', 'IWA', 
             'LBH', 'LBD', 'LBA', 
             'SBH', 'SBD', 'SBA', 
             'WHH', 'WHD', 'WHA', 
             'SYH', 'SYD', 'SYA', 
             'SOH', 'SOD', 'SOA',
             'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 
             'GBAHH', 'GBAHA', 'GBAH', 
             'LBAHH', 'LBAHA', 'LBAH', 
             'Bb1X2', 'BbOU', 'BbAH', 'BbMxAHH', 'BbMxAHA',
             'BWH', 'BWD', 'BWA', 
             'SJH', 'SJD', 'SJA', 
             'VCH', 'VCD', 'VCA',
             'BSH', 'BSD', 'BSA', 
             'Time',
             'BWCH', 'BWCD', 'BWCA', 
             'IWCH', 'IWCD', 'IWCA', 
             'WHCH', 'WHCD', 'WHCA', 
             'VCCH', 'VCCD', 'VCCA',
             '1XBH', '1XBD', '1XBA', 
             '1XBCH', '1XBCD', '1XBCA', 
             'BFH', 'BFD', 'BFA', 
             'BFEH', 'BFED', 'BFEA', 
             'BFE>2.5', 'BFE<2.5', 
             'BFEAHH', 'BFEAHA',
             'BFDH', 'BFDD', 'BFDA',
             'BMGMH','BMGMD','BMGMA',
             'BVH','BVD','BVA',
             'CLH','CLD','CLA',
             'BFDCH','BFDCD','BFDCA',
             'BMGMCH','BMGMCD','BMGMCA',
             'BVCH','BVCD','BVCA',
             'CLCH','CLCD','CLCA',
             'LBCH', 'LBCD','LBCA']

In [10]:
revised_df = original_df.drop(columns=removable, errors='ignore')
print(f"Revised shape after filtering of columns = {revised_df.shape}")

Revised shape after filtering of columns = (9471, 111)


## III. Feature Engineering (I)
#### In this step, certain features will undergo engineered transformations to add more quality to the features

#### Features involving Betting Odds must be converted to Implied Probability and then further normalized, requiring intensive feature engineering 

In [11]:
# 1. Calculate Implied Probabilities (IP) using PSCH/D/A
# We need these temporary IP columns to compute the margin
revised_df = revised_df.copy()

revised_df['IP_Home_PS'] = 1 / revised_df['PSCH']
revised_df['IP_Draw_PS'] = 1 / revised_df['PSCD']
revised_df['IP_Away_PS'] = 1 / revised_df['PSCA']

# 2. Calculate Market Margin
# Margin is the sum of Implied Probabilities minus 1.
revised_df['NormIP_Margin'] = (revised_df['IP_Home_PS'] + revised_df['IP_Draw_PS'] + revised_df['IP_Away_PS']) - 1

# 3. Cleanup: Drop the temporary IP columns
# The NormIP_Margin feature is now permanently added to the DataFrame.
revised_df = revised_df.drop(columns=['IP_Home_PS', 'IP_Draw_PS', 'IP_Away_PS'], errors='ignore')

### Check the final list of odds and only engineer those features

In [12]:
# preparing a list of columns that needs to be engineered and transformed
cols = list(revised_df.columns)

# Odds -> IP -> Normalization

# 1. Classification Odds
open_market = ['BbAvH', 'BbAvD', 'BbAvA'] # opening market average odds (consensus price when the market opens
close_market_classify = ['AvgCH', 'AvgCD', 'AvgCA'] # closing market average odds (consensus price at kick-off) (critical for classification)
pinnacle_classification = ['PSCH', 'PSCD', 'PSCA'] # sharp closing odds by pinnacle
newly_added_odds = ['MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365H', 'B365D', 'B365A', 'B365CH', 'B365CD', 'B365CA']

# 2. Regression Odds
close_market_goals = ['AvgC>2.5', 'AvgC<2.5'] # closing market average odds (critical for goal prediction) (over or under 2.5 goals)
pinnacle_regression = ['PC>2.5', 'PC<2.5'] # sharp closing odds by pinnacle

# asian handicap odds specialize in the elimination of the possibility of a match being drawn. they do not require normalization
# it does by applying a handicap (goal deficit or advantage) to one or more teams, right before kick-off
closing_average = ['AvgCAHH', 'AvgCAHA'] # closing market average odds
pinnacle_asian = ['PCAHH', 'PCAHA'] # asian handicap odds by pinnacle

prob_norm_odds = [open_market, close_market_classify, pinnacle_classification,
                  newly_added_odds, close_market_goals, pinnacle_regression]

prob_only_odds = [closing_average, pinnacle_asian]

In [13]:
# for remaining odds
def Probability_Normalization(temp_df, columns):
    ip_columns = [f'IP_{col}' for col in columns]
    for odd, new in zip(columns, ip_columns):
        temp_df[new] = 1 / temp_df[odd]
    total = temp_df[ip_columns].sum(axis=1)
    norm_columns = [f'NormIP_{col}' for col in columns]
    for ip, norm in zip(ip_columns, norm_columns):
        temp_df[norm] = temp_df[ip] / total
    temp_df = temp_df.drop(columns=columns + ip_columns)
    return temp_df

In [14]:
# only for asian handicap odds 
def Probability(temp_df, columns):
    ip_columns = [f'IP_AHO_{col}' for col in columns]
    for odd, new in zip(columns, ip_columns):
        temp_df[new] = 1 / temp_df[odd]
    temp_df = temp_df.drop(columns=columns)
    return temp_df

In [15]:
df = revised_df.copy()
print(f"Shape: {df.shape}")

Shape: (9471, 112)


### Forming new columns, based on Lag & Rolling features and statistics

#### 1. TEAM SPECIFIC - GOALS FOR & GOALS AGAINST
##### 'Team_GF_L10' = Average Goals scored by the team in the last 10 matches
##### 'Team_GA_L10' = Average Goals conceded by the team in the last 10 matches

#### 2. HOME - GOALS FOR & GOALS AGAINST
##### 'HT_AvgGF_L5' = Average Goals scored by the team in its Home ground the last 5 matches
##### 'HT_AvgGA_L5' = Average Goals conceded by the team in its Home ground the last 5 matches
##### 'HT_AvgGD_L5' = Average Goal difference by the team in its Home ground the last 5 matches

#### 3. AWAY- GOALS FOR & GOALS AGAINST
##### 'AT_AvgGF_L5' = Average Goals scored by the team in Away ground in the last 5 matches 
##### 'AT_AvgGA_L5' = Average Goals conceded by the team in Away ground in the last 5 matches
##### 'AT_AvgGD_L5' = Average Goal difference by the team in Away ground in the last 5 matches

#### 4. HOME TEAM & AWAY TEAM - CLEAN SHEET RECORD (LAST 5 MATCHES)
##### 'HT_CS_L5' = Proportion of Home Team Clean Sheets in the last 5 matches (no. of clean sheets in L5 / 5)
##### 'AT_CS_L5' = Proportion of Away Team Clean Sheets in the last 5 matches (no. of clean sheets in L5 / 5)

#### 5. HOME TEAM & AWAY TEAM - AVERAGE SHOTS (LAST 5 MATCHES)
##### 'HT_AvgShots_L5' = Average Shots taken by the Home Team in the last 5 matches
##### 'AT_AvgShots_L5' = Average Shots taken by the Away Team in the Last 5 matches

#### 6. HOME TEAM & AWAY TEAM - SHOT ACCURACY (LAST 5 MATCHES)
##### 'HT_ShotAccuracy_L5' = 'HST' / 'HS' (shots on target / total shots taken) (home team)
##### 'AT_ShotAccuracy_L5' = 'AST' / 'AS' (shots on target / total shots taken) (away team)

#### 7. HOME TEAM & AWAY TEAM - SHOT CONVERSION (LAST 5 MATCHES) 
##### 'HT_ShotConversion_L5' = Goals scored / Total shots taken, by the Home Team (last 5 matches)
##### 'AT_ShotConversion_L5' = Goals scored / Total shots taken, by the Away Team (last 5 matches)

#### 8. HOME TEAM & AWAY TEAM - WIN PROPORTION (LAST 5 MATCHES)
##### 'HT_WinRate_L5' = Win Proportion of Home Team in the last 5 matches (no. of wins in L5 / 5)
##### 'AT_WinRate_L5' = Win Proportion of Away Team in the last 5 matches (no. of wins in L5 / 5)

#### 9. TOTAL AVERAGE CARDS GIVEN BY THE REFEREE
##### 'Ref_Avg_Cards' = Average number of Yellow & Red cards given by the referee prior to the match

#### 10. INDICATOR FOR THE LAST WIN OF HOME TEAM & AWAY TEAM

#### Storing the dataset in ascending order for Lag & Rolling features 

In [16]:
def DateConversion(col):
    date = str(col).strip()
    if len(date.split("/")[-1]) == 2:
        return pd.to_datetime(col, format="%d/%m/%y", dayfirst=True)
    else: return pd.to_datetime(col, format="%d/%m/%Y", dayfirst=True)

In [17]:
df['Date'] = df['Date'].apply(DateConversion)
df = df.sort_values('Date', ascending=True).reset_index(drop=True)

#### 1. TEAM SPECIFIC - GF & GA

In [18]:
# dataframe consisting of home team statistics
home_df = df[['Date', 'HomeTeam', 'FTHG', 'FTAG']].copy()
home_df.columns = ['Date', 'Team', 'GoalsFor', 'GoalsAgainst'] # renaming the home_df columns
home_df['Venue'] = 'Home'

# dataframe consisting of away team statistics
away_df = df[['Date', 'AwayTeam', 'FTAG', 'FTHG']].copy()
away_df.columns = ['Date', 'Team', 'GoalsFor', 'GoalsAgainst'] # renaming the away_df columns
away_df['Venue'] = 'Away'

# combining both the dataframes
combined_df = pd.concat([home_df, away_df], ignore_index=True)
combined_df = combined_df.sort_values(['Team', 'Date']).reset_index(drop=True)

# compute the total goals scored by the team in the past 5 matches 
combined_df['Team_GF_L5'] = combined_df.groupby('Team')['GoalsFor'].transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())

# compute the total goals conceded by the team in the past 5 matches
combined_df['Team_GA_L5'] = combined_df.groupby('Team')['GoalsAgainst'].transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())

# home teams
df = df.merge(combined_df[['Date', 'Team', 'Team_GF_L5', 'Team_GA_L5']], 
             left_on=['Date', 'HomeTeam'], 
             right_on=['Date', 'Team'],  
             how='left') 
df = df.rename(columns={'Team_GF_L5': 'HT_AvgGF_L5', 'Team_GA_L5': 'HT_AvgGA_L5'})
df = df.drop(columns='Team') 

# away teams
df = df.merge(combined_df[['Date', 'Team', 'Team_GF_L5', 'Team_GA_L5']],
             left_on=['Date', 'AwayTeam'],
             right_on=['Date', 'Team'],
             how='left') # keep all rows of main df, even if combined_df consists of some NaN values
df = df.rename(columns={'Team_GF_L5': 'AT_AvgGF_L5', 'Team_GA_L5': 'AT_AvgGA_L5'})
df = df.drop(columns='Team')

In [19]:
print(f"Shape (after 1st stage of feature engineering) = {df.shape}")

Shape (after 1st stage of feature engineering) = (9474, 116)


#### 2. HOME GROUND SPECIFICS - GOALS FOR & GOALS AGAINST & GOAL DIFFERENCE

In [20]:
home_ground = df[['Date', 'HomeTeam', 'FTHG', 'FTAG']].copy()
home_ground = home_ground.rename(columns={'HomeTeam':'Team', 'FTHG':'GoalsFor', 'FTAG':'GoalsAgainst'})
home_ground = home_ground.sort_values(['Team', 'Date']).reset_index(drop=True)
home_ground['HG_HT_AvgGF_L5'] = home_ground.groupby('Team')['GoalsFor'].transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())
home_ground['HG_HT_AvgGA_L5'] = home_ground.groupby('Team')['GoalsAgainst'].transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())
home_ground['HG_HT_AvgGD_L5'] = home_ground['HG_HT_AvgGF_L5'] - home_ground['HG_HT_AvgGA_L5']
df = df.merge(home_ground[['Date', 'Team', 'HG_HT_AvgGF_L5', 'HG_HT_AvgGA_L5', 'HG_HT_AvgGD_L5']],
             left_on=['Date', 'HomeTeam'],
             right_on=['Date', 'Team'],
             how='left')
df = df.drop(columns='Team')

#### 3. AWAY GROUND SPECIFICS - GOALS FOR & GOALS AGAINST & GOAL DIFFERENCE

In [21]:
away_ground = df[['Date', 'AwayTeam', 'FTHG', 'FTAG']].copy()
away_ground = away_ground.rename(columns={'AwayTeam':'Team', 'FTHG':'GoalsAgainst', 'FTAG':'GoalsFor'})
away_ground = away_ground.sort_values(['Team', 'Date']).reset_index(drop=True)
away_ground['AG_AT_AvgGF_L5'] = away_ground.groupby('Team')['GoalsFor'].transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())
away_ground['AG_AT_AvgGA_L5'] = away_ground.groupby('Team')['GoalsAgainst'].transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())
away_ground['AG_AT_AvgGD_L5'] = away_ground['AG_AT_AvgGF_L5'] - away_ground['AG_AT_AvgGA_L5']
df = df.merge(away_ground[['Date', 'Team', 'AG_AT_AvgGF_L5', 'AG_AT_AvgGA_L5', 'AG_AT_AvgGD_L5']],
             left_on=['Date', 'AwayTeam'],
             right_on=['Date', 'Team'],
             how='left')
df = df.drop(columns='Team')

In [22]:
print(f"Shape (after 2nd & 3rd stages of feature engineering) = {df.shape}")

Shape (after 2nd & 3rd stages of feature engineering) = (9726, 122)


In [23]:
df = df[~df.duplicated()].reset_index(drop=True)

In [24]:
print(f"Shape (after removing duplicates) = {df.shape}")

Shape (after removing duplicates) = (9471, 122)


#### 4. AVERAGE SHOTS FOR HOME TEAM & AWAY TEAM

In [25]:
shots_home = df[['Date','HomeTeam','HS']].copy()
shots_home.columns = ['Date','Team','Shots']
shots_home['Venue'] = 'Home'

shots_away = df[['Date','AwayTeam','AS']].copy()
shots_away.columns = ['Date','Team','Shots']
shots_away['Venue'] = 'Away'

shots_df = pd.concat([shots_home, shots_away], ignore_index=True)
shots_df = shots_df.sort_values(['Team','Date']).reset_index(drop=True)
shots_df['Shots_L5'] = (shots_df.groupby('Team')['Shots'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean()))

shots_df = shots_df[['Date','Team','Shots_L5']]
shots_df = shots_df.drop_duplicates(subset=['Date','Team'])

mapping = dict(zip(zip(shots_df['Date'], shots_df['Team']), shots_df['Shots_L5']))

df['HT_AvgShots_L5'] = df.apply(lambda x: mapping.get((x['Date'], x['HomeTeam'])), axis=1)
df['AT_AvgShots_L5'] = df.apply(lambda x: mapping.get((x['Date'], x['AwayTeam'])), axis=1)

In [26]:
print(f"Shape (after 4th stage of feature engineering) = {df.shape}")

Shape (after 4th stage of feature engineering) = (9471, 124)


#### 5. SHOT ACCURACY - HOME TEAM & AWAY TEAM

In [27]:
home_shots = df[['Date', 'HomeTeam', 'HS', 'HST']].copy()
home_shots = home_shots.rename(columns={'HomeTeam':'Team'})
home_shots['Shot_Accuracy'] = home_shots['HST'] / home_shots['HS']
home_shots = home_shots[['Date', 'Team', 'Shot_Accuracy']]

away_shots = df[['Date', 'AwayTeam', 'AS', 'AST']].copy()
away_shots = away_shots.rename(columns={'AwayTeam':'Team'})
away_shots['Shot_Accuracy'] = away_shots['AST'] / away_shots['AS']
away_shots = away_shots[['Date', 'Team', 'Shot_Accuracy']]

shots_acc_df = pd.concat([home_shots, away_shots], ignore_index=True)
shots_acc_df = shots_acc_df.sort_values(['Team', 'Date']).reset_index(drop=True)

shots_acc_df['ShotAccuracy_L5'] = shots_acc_df.groupby('Team')['Shot_Accuracy'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())

shots_acc_df = shots_acc_df[['Date','Team','ShotAccuracy_L5']].drop_duplicates()

mapping = dict(zip(zip(shots_acc_df['Date'], shots_acc_df['Team']), shots_acc_df['ShotAccuracy_L5']))

df['HT_ShotAccuracy_L5'] = df.apply(lambda x: mapping.get((x['Date'], x['HomeTeam'])), axis=1)
df['AT_ShotAccuracy_L5'] = df.apply(lambda x: mapping.get((x['Date'], x['AwayTeam'])), axis=1)

In [28]:
print(f"Shape (after 5th stage of feature engineering) = {df.shape}")

Shape (after 5th stage of feature engineering) = (9471, 126)


#### 6. SHOT CONVERSION - HOME TEAM & AWAY TEAM

In [29]:
ht_shots = df[['Date', 'HomeTeam', 'FTHG', 'HS']].copy()
ht_shots = ht_shots.rename(columns={'HomeTeam':'Team'})
ht_shots['Shot_Conversion'] = np.where(ht_shots['HS'] > 0, ht_shots['FTHG'] / ht_shots['HS'], 0)
ht_shots = ht_shots[['Date', 'Team', 'Shot_Conversion']]

at_shots = df[['Date', 'AwayTeam', 'FTAG', 'AS']].copy()
at_shots = at_shots.rename(columns={'AwayTeam':'Team'})
at_shots['Shot_Conversion'] = np.where(at_shots['AS'] > 0, at_shots['FTAG'] / at_shots['AS'], 0)
at_shots = at_shots[['Date', 'Team', 'Shot_Conversion']]

shots_conv_df = pd.concat([ht_shots, at_shots], ignore_index=True)
shots_conv_df = shots_conv_df.sort_values(['Team', 'Date']).reset_index(drop=True)

shots_conv_df['ShotConversion_L5'] = shots_conv_df.groupby('Team')['Shot_Conversion'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())

shots_conv_df = shots_conv_df[['Date','Team','ShotConversion_L5']].drop_duplicates()

mapping = dict(zip(zip(shots_conv_df['Date'], shots_conv_df['Team']), shots_conv_df['ShotConversion_L5']))

df['HT_ShotConversion_L5'] = df.apply(lambda x: mapping.get((x['Date'], x['HomeTeam'])), axis=1)
df['AT_ShotConversion_L5'] = df.apply(lambda x: mapping.get((x['Date'], x['AwayTeam'])), axis=1)

In [30]:
print(f"Shape (after 6th stage of feature engineering) = {df.shape}")

Shape (after 6th stage of feature engineering) = (9471, 128)


#### 7. CLEAN SHEET RECORD - HOME TEAM & AWAY TEAM

In [31]:
home_record = df[['Date', 'HomeTeam', 'FTAG']].copy()
home_record = home_record.rename(columns={'HomeTeam':'Team', 'FTAG':'GoalsAgainst'})
home_record['Clean_Sheet'] = np.where(home_record['GoalsAgainst'] > 0, 0, 1)
home_record = home_record[['Date', 'Team', 'Clean_Sheet']]

away_record = df[['Date', 'AwayTeam', 'FTHG']].copy()
away_record = away_record.rename(columns={'AwayTeam':'Team', 'FTHG':'GoalsAgainst'})
away_record['Clean_Sheet'] = np.where(away_record['GoalsAgainst'] > 0, 0, 1)
away_record = away_record[['Date', 'Team', 'Clean_Sheet']]

cs_record = pd.concat([home_record, away_record], ignore_index=True)
cs_record = cs_record.sort_values(['Team', 'Date']).reset_index(drop=True)

cs_record['CleanSheet_L5'] = cs_record.groupby('Team')['Clean_Sheet'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())
cs_record = cs_record[['Date', 'Team', 'CleanSheet_L5']].drop_duplicates()

cs_mapping = dict(zip(zip(cs_record['Date'], cs_record['Team']), cs_record['CleanSheet_L5']))

df['HT_CS_L5'] = df.apply(lambda x: cs_mapping.get((x['Date'], x['HomeTeam'])), axis=1)
df['AT_CS_L5'] = df.apply(lambda x: cs_mapping.get((x['Date'], x['AwayTeam'])), axis=1)

In [32]:
print(f"Shape (after 7th stage of feature engineering) = {df.shape}")

Shape (after 7th stage of feature engineering) = (9471, 130)


#### 8. WIN RATE - HOME TEAM & AWAY TEAM

In [33]:
home_wins = df[['Date', 'HomeTeam', 'FTR']].copy()
home_wins = home_wins.rename(columns={'HomeTeam':'Team'})
home_wins['Win'] = np.where(home_wins['FTR'] == 'H', 1, 0)
home_wins = home_wins[['Date', 'Team', 'Win']]

away_wins = df[['Date', 'AwayTeam', 'FTR']].copy()
away_wins = away_wins.rename(columns={'AwayTeam':'Team'})
away_wins['Win'] = np.where(away_wins['FTR'] == 'A', 1, 0)
away_wins = away_wins[['Date', 'Team', 'Win']]

wins_record = pd.concat([home_wins, away_wins], ignore_index=True)
wins_record = wins_record.sort_values(['Team', 'Date']).reset_index(drop=True)

wins_record['Wins_L5'] = wins_record.groupby('Team')['Win'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())
wins_record = wins_record[['Date', 'Team', 'Wins_L5']].drop_duplicates()

win_mapping = dict(zip(zip(wins_record['Date'], wins_record['Team']), wins_record['Wins_L5']))

df['HT_WinRate_L5'] = df.apply(lambda x: win_mapping.get((x['Date'], x['HomeTeam'])), axis=1)
df['AT_WinRate_L5'] = df.apply(lambda x: win_mapping.get((x['Date'], x['AwayTeam'])), axis=1)

In [34]:
print(f"Shape (after 8th stage of feature engineering) = {df.shape}")

Shape (after 8th stage of feature engineering) = (9471, 132)


#### 9. TOTAL AVERAGE CARDS GIVEN BY THE REFEREE PRIOR TO THE MATCH

In [35]:
df['TotalCards'] = (df['HY'] + df['AY'] + df['HR'] + df['AR'])
df['Ref_Avg_Cards'] = df.groupby('Referee')['TotalCards'].transform(lambda x: x.expanding().mean().shift(1))
df['Ref_Avg_Cards'] = df['Ref_Avg_Cards'].fillna(df['TotalCards'].median())
df = df.drop(columns='TotalCards', errors='ignore')

In [36]:
print(f"Shape (after 9th stage of feature engineering) = {df.shape}")

Shape (after 9th stage of feature engineering) = (9471, 133)


#### 10. INDICATOR FOR THE LAST WIN OF HOME TEAM & AWAY TEAM

In [37]:
df['HT_Win_Indicator'] = (df['FTR'] == 'H').astype(int)
df['AT_Win_Indicator'] = (df['FTR'] == 'A').astype(int)

ht_wins = df[['Date', 'HomeTeam', 'HT_Win_Indicator']].rename(columns={'HomeTeam':'Team', 'HT_Win_Indicator':'Win'})
at_wins = df[['Date', 'AwayTeam', 'AT_Win_Indicator']].rename(columns={'AwayTeam':'Team', 'AT_Win_Indicator':'Win'})

combined = pd.concat([ht_wins, at_wins], ignore_index=True)
combined = combined.sort_values(by=['Team', 'Date']).reset_index(drop=True)

combined['Last_Win'] = combined.groupby('Team')['Win'].shift(1)

df = pd.merge(
    df,
    combined[['Team', 'Date', 'Last_Win']],
    left_on=['HomeTeam', 'Date'],
    right_on=['Team', 'Date'],
    how='left'
).rename(columns={'Last_Win': 'HT_Last_Win'}).drop(columns=['Team'])

df = pd.merge(
    df,
    combined[['Team', 'Date', 'Last_Win']],
    left_on=['AwayTeam', 'Date'],
    right_on=['Team', 'Date'],
    how='left'
).rename(columns={'Last_Win': 'AT_Last_Win'}).drop(columns=['Team'])

df['HT_Last_Win'] = df['HT_Last_Win'].fillna(0)
df['AT_Last_Win'] = df['AT_Last_Win'].fillna(0)

df = df.drop(columns=['HT_Win_Indicator', 'AT_Win_Indicator'], errors='ignore')

In [38]:
print(f"Shape (after 10th stage of feature engineering) = {df.shape}")

Shape (after 10th stage of feature engineering) = (9474, 135)


In [39]:
df = df.drop_duplicates()

In [40]:
print(f"Shape (after 10th stage of feature engineering and dropping duplicates) = {df.shape}")

Shape (after 10th stage of feature engineering and dropping duplicates) = (9471, 135)


## IV. Data Cleaning

#### 1. Deleting row having every value as NaN

In [41]:
df.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Attendance,...,AT_ShotAccuracy_L5,HT_ShotConversion_L5,AT_ShotConversion_L5,HT_CS_L5,AT_CS_L5,HT_WinRate_L5,AT_WinRate_L5,Ref_Avg_Cards,HT_Last_Win,AT_Last_Win
9466,2025-09-27,Tottenham,Wolves,1.0,1.0,D,0.0,0.0,D,NaN,...,0.316667,0.150054,0.045833,0.6,0.0,0.6,0.0,3.812500,0.0,0.0
9467,2025-09-28,Newcastle,Arsenal,1.0,2.0,A,1.0,0.0,H,NaN,...,0.252904,0.052500,0.131944,0.8,0.6,0.2,0.6,3.769231,0.0,0.0
9468,2025-09-28,Aston Villa,Fulham,3.0,1.0,H,1.0,1.0,D,NaN,...,0.336084,0.016667,0.126813,0.4,0.2,0.0,0.4,3.419643,0.0,1.0
9469,2025-09-29,Everton,West Ham,1.0,1.0,D,1.0,0.0,H,NaN,...,0.455952,0.118586,0.091667,0.4,0.2,0.4,0.2,4.170732,0.0,0.0
9470,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,0.0,0.0


In [42]:
df = df[:-1]

In [43]:
df.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Attendance,...,AT_ShotAccuracy_L5,HT_ShotConversion_L5,AT_ShotConversion_L5,HT_CS_L5,AT_CS_L5,HT_WinRate_L5,AT_WinRate_L5,Ref_Avg_Cards,HT_Last_Win,AT_Last_Win
9465,2025-09-27,Crystal Palace,Liverpool,2.0,1.0,H,1.0,0.0,H,NaN,...,0.416105,0.147222,0.228099,0.6,0.4,0.4,1.0,3.541667,1.0,1.0
9466,2025-09-27,Tottenham,Wolves,1.0,1.0,D,0.0,0.0,D,NaN,...,0.316667,0.150054,0.045833,0.6,0.0,0.6,0.0,3.812500,0.0,0.0
9467,2025-09-28,Newcastle,Arsenal,1.0,2.0,A,1.0,0.0,H,NaN,...,0.252904,0.052500,0.131944,0.8,0.6,0.2,0.6,3.769231,0.0,0.0
9468,2025-09-28,Aston Villa,Fulham,3.0,1.0,H,1.0,1.0,D,NaN,...,0.336084,0.016667,0.126813,0.4,0.2,0.0,0.4,3.419643,0.0,1.0
9469,2025-09-29,Everton,West Ham,1.0,1.0,D,1.0,0.0,H,NaN,...,0.455952,0.118586,0.091667,0.4,0.2,0.4,0.2,4.170732,0.0,0.0


#### 2. Handling sparse & scattered NaN values

In [44]:
# Identifying columns having NaN values
null_columns = {key:value for key, value in dict(df.isnull().sum()).items()}

STAGE 1: DELETION OF COLUMNS NOT SATISFYING THRESHOLD RANGE

In [45]:
# "Must-Delete" columns : Columns containing more than 90% NaN values
rows = (df.shape)[0]
threshold = 0.90
must_delete_cols = {key: value for key, value in null_columns.items() if value > (threshold * rows)}
print(f"Must-Delete columns: {must_delete_cols}")
print(f"Length: {len(must_delete_cols)}")

Must-Delete columns: {'Attendance': 8711, 'HBP': 8710, 'ABP': 8710, 'B365AH': 8817, 'BFCH': 9090, 'BFCD': 9090, 'BFCA': 9090, 'BFECH': 9030, 'BFECD': 9030, 'BFECA': 9030, 'BFEC>2.5': 9030, 'BFEC<2.5': 9030, 'BFECAHH': 9030, 'BFECAHA': 9030}
Length: 14


In [46]:
df_stage1 = df.drop(columns=list(must_delete_cols.keys()), axis=1)

print(f"Shape after 1st round of deletion: {df_stage1.shape}")
print(f"Length of 'Must-Delete' columns: {len(must_delete_cols)}")
print(f"Shape of Original Data frame: {df.shape}")

after_stage1_null_columns = {key:value for key, value in dict(df_stage1.isnull().sum()).items()}

Shape after 1st round of deletion: (9470, 121)
Length of 'Must-Delete' columns: 14
Shape of Original Data frame: (9470, 135)


STAGE 2: IMPUTING THE NORMALIZED IP COLUMNS (FILLING NULL VALUES)

In [47]:
# Imputable columns : Columns containing less than 95% NaN values requires imputing and filling of NaN values
imputable_cols = {key: value for key, value in after_stage1_null_columns.items() if value < (threshold * rows) and value != 0}
df_stage2 = df_stage1.copy()

In [48]:
fixed_cols = ['Date', 'HomeTeam', 'AwayTeam', 'Referee', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR']
exclude_cols = fixed_cols + [col for col in df_stage2.columns if col.startswith(('HT_', 'AT_', 'HG_', 'AG_'))]

for col in list(imputable_cols.keys()):
    if col not in exclude_cols: df_stage2[col] = df_stage2[col].fillna(df_stage2[col].median())
    
after_stage2_null_columns = {key:value for key, value in dict(df_stage2.isnull().sum()).items()}

STAGE 3: FILLING NULL VALUES OF ROLLING LAG FEATURES AS 0 

In [49]:
rolling_cols = {key: value for key, value in after_stage2_null_columns.items() if value > 0}

df_stage3 = df_stage2.copy()

for col in rolling_cols:
    df_stage3[col] = df_stage3[col].fillna(0)
    
after_stage3_null_columns = {key:value for key, value in dict(df_stage3.isnull().sum()).items()}

FINAL STAGE: CONVERTING THE RESULTANT DATA FRAME INTO OUR ORIGINAL ONE

In [50]:
df = df_stage3.copy()
print(f"Final Shape of the Dataset: {df.shape}")

Final Shape of the Dataset: (9470, 121)


In [51]:
for category in prob_norm_odds:
    df = Probability_Normalization(df, category) 
for category in prob_only_odds:
    df = Probability(df, category)

## V. Feature Engineering (II)
#### At this step, we add some more derived and engineered features using the existing rolling features

#### 1. Ground-Specific Goal Difference (Home Team's strength at Home Ground v/s Away Team's strngth at Away Ground)

In [52]:
df['GD_Diff_L5'] = df['HG_HT_AvgGD_L5'] - df['AG_AT_AvgGD_L5']

#### 2. Attack v/s Defense (Home Team's Offensive Strength at Home Ground v/s Away Team's Defensive Strength at Away Ground)

In [53]:
df['Attack_Defense_L5'] = df['HG_HT_AvgGF_L5'] - df['AG_AT_AvgGA_L5']

#### 3. Overall Win Rate Difference (Home Team's Win Rate v/s Away Team's Win Rate)

In [54]:
df['Overall_Win_Rate_L5'] = df['HT_WinRate_L5'] - df['AT_WinRate_L5']

#### 4. Overall Shot Conversion Difference (Home Team's Shot Conversion Rate v/s Away Team's Shot Conversion Rate)

In [55]:
df['ShotConversion_Diff_L5'] = df['HT_ShotConversion_L5'] - df['AT_ShotConversion_L5']

In [56]:
print(f"Final Shape of the Dataset: {df.shape}")

Final Shape of the Dataset: (9470, 125)


#### 5. Head-to-Head Points (Home Team H2H Points with Away Team)

In [57]:
df = df.sort_values(by='Date')
df['HT_Points'] = np.where(df['FTR'] == 'H', 3, np.where(df['FTR'] == 'D', 1, 0))
df['AT_Points'] = np.where(df['FTR'] == 'A', 3, np.where(df['FTR'] == 'D', 1, 0))

teams = df[['HomeTeam', 'AwayTeam']].values # creates a 2D numpy array
teams.sort(axis=1) # sort the teams alphabetically

df['MatchUp'] = teams[:,0] + "_" + teams[:,1]
df['H2H_HT_Points_L5'] = df.groupby('MatchUp')['HT_Points'].transform(lambda x: x.shift(1).rolling(5, min_periods=1).sum())
df['H2H_AT_Points_L5'] = df.groupby('MatchUp')['AT_Points'].transform(lambda x: x.shift(1).rolling(5, min_periods=1).sum())

df['H2H_HT_Points_L5'] = df['H2H_HT_Points_L5'].fillna(0)
df['H2H_AT_Points_L5'] = df['H2H_AT_Points_L5'].fillna(0)

df['H2H_Points_Diff'] = df['H2H_HT_Points_L5'] - df['H2H_AT_Points_L5']

In [58]:
df = df.drop(columns=['HT_Points', 'AT_Points', 'MatchUp'], errors='ignore')

In [59]:
print(f"Final Shape of the Dataset: {df.shape}")

Final Shape of the Dataset: (9470, 128)


In [60]:
dftwoszn = df.tail(440).reset_index(drop=True).copy()
dftwoszn.loc[:379, 'season'] = 2024
dftwoszn.loc[380:, 'season'] = 2025
dftwoszn['season'] = dftwoszn['season'].astype(int)

In [61]:
dftwoszn.shape

(440, 129)

## VI. Working with 24/25 and 25/26 data ONLY

In [62]:
fe_gk_stats = pd.read_pickle('C:/exp1/Pickle Files/fe_gk_stats.pkl')
fe_def_stats = pd.read_pickle('C:/exp1/Pickle Files/fe_def_stats.pkl')
fe_mid_stats = pd.read_pickle('C:/exp1/Pickle Files/fe_mid_stats.pkl')
fe_fwd_stats = pd.read_pickle('C:/exp1/Pickle Files/fe_fwd_stats.pkl')
teams_matches = pd.read_pickle('C:/exp1/Pickle Files/teams_matches.pkl') 
print("All the new datasets have been loaded and ready for merging.")
print(f"Shape of GK stats: {fe_gk_stats.shape}")
print(f"Shape of DEF stats: {fe_def_stats.shape}")
print(f"Shape of MID stats: {fe_mid_stats.shape}")
print(f"Shape of FWD stats: {fe_fwd_stats.shape}")
print(f"Shape of Teams+Matches stats: {teams_matches.shape}")

All the new datasets have been loaded and ready for merging.
Shape of GK stats: (892, 25)
Shape of DEF stats: (3378, 41)
Shape of MID stats: (3992, 59)
Shape of FWD stats: (851, 51)
Shape of Teams+Matches stats: (440, 115)


### Merging the current dataset with the new data of Teams + Matches

#### Since merge will be done on Date, Home Team and Away Team: It is ensure they are of the same type and values

In [63]:
teams_matches['kickoff_time'] = teams_matches['kickoff_time'].dt.normalize()
teams_matches = teams_matches.rename(columns={'kickoff_time':'Date'}, errors='ignore')

In [64]:
teams_matches['HT_name'] = teams_matches['HT_name'].replace({'Man Utd':'Man United', 'Spurs':'Tottenham'})
teams_matches['AT_name'] = teams_matches['AT_name'].replace({'Man Utd':'Man United', 'Spurs':'Tottenham'})
teams_matches = teams_matches.rename(columns={'HT_name':'HomeTeam', 'AT_name':'AwayTeam'}, errors='ignore')

#### It was also found that for season 2025: the dates for the following matches were imputed incorrectly
#### Match 1: Brentford vs Aston Villa (23/08/2025) [Incorrect date: 16/09/2025]
#### Match 2: Wolves vs Everton (30/08/2025) [Incorrect date: 23/09/2025]

In [65]:
home1 = 'Brentford'
away1 = 'Aston Villa'
m1_date = '2025-08-23'
m1_faulty = '2025-09-16'

home2 = 'Wolves'
away2 = 'Everton'
m2_date = '2025-08-30'
m2_faulty = '2025-09-23'

teams_matches.loc[(teams_matches['Date'] == m1_faulty) & (teams_matches['HomeTeam'] == home1), 'Date'] = m1_date
teams_matches.loc[(teams_matches['Date'] == m2_faulty) & (teams_matches['HomeTeam'] == home2), 'Date'] = m2_date

In [66]:
teams_matches.shape

(440, 115)

In [67]:
merged_stage1 = dftwoszn.merge(
    teams_matches,
    on=['Date', 'season', 'HomeTeam', 'AwayTeam'],
    how='left',
    suffixes=('', '_tm')
)
merged_stage1 = merged_stage1.drop(columns=[col for col in merged_stage1.columns if col.endswith('_tm')], errors='ignore')
print("Data has been merged successfully!")

Data has been merged successfully!


In [68]:
print(f"Shape of Merged Data: {merged_stage1.shape}")

Shape of Merged Data: (440, 240)


In [69]:
raw_stats = ['HTHG', 'HTAG', 'HTR', 'Referee', 'home_score', 'away_score', 'gameweek', 'finished', 'match_url',
       'home_total_shots', 'away_total_shots', 'home_shots_on_target', 'away_shots_on_target',
       'home_fouls_committed', 'away_fouls_committed', 'home_corners', 'away_corners',
       'home_yellow_cards', 'away_yellow_cards', 'home_red_cards', 'away_red_cards',
       'fotmob_id', 'stats_processed', 'player_stats_processed']

odds = [
    'GBH', 'GBD', 'GBA', 'BbMxH', 'BbMxD', 'BbMxA',
    'GB>2.5', 'GB<2.5', 'B365>2.5', 'B365<2.5', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'P>2.5', 'P<2.5',
    'B365AHH', 'B365AHA', 'BbAHh', 'BbAvAHH', 'BbAvAHA', 'AHh', 'PAHH', 'PAHA',
    'NormIP_BbAvH', 'NormIP_BbAvD', 'NormIP_BbAvA',
    'NormIP_MaxH', 'NormIP_MaxD', 'NormIP_MaxA', 'NormIP_AvgH', 'NormIP_AvgD', 'NormIP_AvgA', 
    'NormIP_B365H', 'NormIP_B365D', 'NormIP_B365A', 
    'IP_AHO_AvgCAHH', 'IP_AHO_AvgCAHA', 'IP_AHO_PCAHH', 'IP_AHO_PCAHA',
]

match_stats = [
    'home_passes', 'away_passes', 'home_accurate_passes', 'away_accurate_passes', 
    'home_accurate_passes_pct', 'away_accurate_passes_pct', 
    'home_shots_off_target', 'away_shots_off_target', 
    'home_blocked_shots', 'away_blocked_shots', 
    'home_hit_woodwork', 'away_hit_woodwork',
    'home_shots_inside_box', 'away_shots_inside_box', 
    'home_shots_outside_box', 'away_shots_outside_box',
    'home_own_half', 'away_own_half', 'home_opposition_half', 'away_opposition_half',
    'home_accurate_long_balls', 'away_accurate_long_balls', 'home_accurate_long_balls_pct', 
    'away_accurate_long_balls_pct', 'home_accurate_crosses', 'away_accurate_crosses', 
    'home_accurate_crosses_pct', 'away_accurate_crosses_pct', 'home_throws', 'away_throws',
    'home_offsides', 'away_offsides', 
    'home_tackles_won', 'away_tackles_won', 'home_tackles_won_pct', 'away_tackles_won_pct', 
    'home_interceptions', 'away_interceptions', 'home_blocks', 'away_blocks', 
    'home_clearances', 'away_clearances', 'home_keeper_saves', 'away_keeper_saves', 
    'home_duels_won', 'away_duels_won', 'home_ground_duels_won', 'away_ground_duels_won', 
    'home_ground_duels_won_pct', 'away_ground_duels_won_pct', 'home_aerial_duels_won', 
    'away_aerial_duels_won', 'home_aerial_duels_won_pct', 'away_aerial_duels_won_pct', 
    'home_successful_dribbles', 'away_successful_dribbles', 'home_successful_dribbles_pct', 
    'away_successful_dribbles_pct',
]

future_cols_to_drop = raw_stats + match_stats + odds
fr_merged_stage1 = merged_stage1.copy()
fr_merged_stage1 = fr_merged_stage1.rename(columns={'home_team':'HT_code', 'away_team':'AT_code'})

### Incorporating & Aggregating Player Match Stats into our Dataset

In [70]:
fr_merged_stage1.dtypes

Date                        datetime64[ns]
HomeTeam                            object
AwayTeam                            object
FTHG                               float64
FTAG                               float64
                                 ...      
AT_strength_overall_away             int64
AT_strength_attack_away              int64
AT_strength_defence_away             int64
AT_elo                               int64
elo_diff                           float64
Length: 240, dtype: object

In [71]:
player_stats_df = {
    'GK':fe_gk_stats,
    'DEF':fe_def_stats,
    'MID':fe_mid_stats,
    'FWD':fe_fwd_stats
}
aggregated_data = []
for position, stats in player_stats_df.items():
    print(f"Aggregating {position} stats...")
    rolling_cols = [col for col in stats.columns if col.startswith('L5_Avg')]
    agg_df = stats.groupby(['match_id', 'team_code'])[rolling_cols].mean().reset_index()
    agg_df.columns = ['match_id', 'team_code'] + [f'{position}_{col}' for col in rolling_cols]
    aggregated_data.append(agg_df)
    print(f"{position} stats aggregated successfully! Shape: {agg_df.shape}\n")
final_players_stats = aggregated_data[0]
for data in aggregated_data[1:]:
    final_players_stats = final_players_stats.merge(
        data,
        on=['match_id', 'team_code'],
        how='outer'
    )
print(f"Final Aggregated Players stats ready! Shape: {final_players_stats.shape}")

Aggregating GK stats...
GK stats aggregated successfully! Shape: (885, 12)

Aggregating DEF stats...
DEF stats aggregated successfully! Shape: (1029, 20)

Aggregating MID stats...
MID stats aggregated successfully! Shape: (1082, 29)

Aggregating FWD stats...
FWD stats aggregated successfully! Shape: (733, 25)

Final Aggregated Players stats ready! Shape: (1255, 80)


In [72]:
final_players_stats = final_players_stats.fillna(0)

In [73]:
final_players_stats.shape

(1255, 80)

### Double-Merge (Home & Away) into our Master dataset

In [74]:
# home team
home_final_players_stats = final_players_stats.copy()
player_features = [col for col in home_final_players_stats.columns if col not in ['match_id', 'team_code']]
home_rename_map = {col: f'HT_TEMP_{col}' for col in player_features}
home_final_players_stats = home_final_players_stats.rename(columns=home_rename_map)

fr_merged_stage2_step1 = fr_merged_stage1.merge(
    home_final_players_stats,
    left_on=['match_id', 'HT_code'],
    right_on=['match_id', 'team_code'],
    how='left'
)

home_players = [col for col in fr_merged_stage2_step1.columns if col.startswith('HT_TEMP_')]
home_rename_map_final = {col: col.replace('HT_TEMP_', 'HT_') for col in home_players}
fr_merged_stage2_step1 = fr_merged_stage2_step1.rename(columns=home_rename_map_final)
fr_merged_stage2_step1 = fr_merged_stage2_step1.drop(columns=['team_code'], errors='ignore')
print("Home Team Player stats merged successfully!")

Home Team Player stats merged successfully!


In [75]:
# away team
away_final_players_stats = final_players_stats.copy()
player_features = [col for col in away_final_players_stats.columns if col not in ['match_id', 'team_code']]
away_rename_map = {col: f'AT_TEMP_{col}' for col in player_features}
away_final_players_stats = away_final_players_stats.rename(columns=away_rename_map)

fr_merged_stage2_step2 = fr_merged_stage2_step1.merge(
    away_final_players_stats,
    left_on=['match_id', 'AT_code'],
    right_on=['match_id', 'team_code'],
    how='left'
)

away_players = [col for col in fr_merged_stage2_step2.columns if col.startswith('AT_TEMP_')]
away_rename_map_final = {col: col.replace('AT_TEMP_', 'AT_') for col in away_players}
fr_merged_stage2_step2 = fr_merged_stage2_step2.rename(columns=away_rename_map_final)
fr_merged_stage2_step2 = fr_merged_stage2_step2.drop(columns=['team_code'], errors='ignore')
print("Away Team Player stats merged successfully!")

Away Team Player stats merged successfully!


#### Check for NaN values in the final merged dataset

In [76]:
# {key: value for key, value in dict(fr_merged_stage2_step2.isnull().sum()).items() if value > 0}

In [77]:
cols_to_impute = [col for col in fr_merged_stage2_step2.columns if col.startswith(('HT_GK_', 'HT_DEF_', 'HT_MID_', 'HT_FWD_', 'AT_GK_', 'AT_DEF_', 'AT_MID_', 'AT_FWD_'))] 
player_stats_feature = [col for col in cols_to_impute]
fr_merged_stage2_step2[player_stats_feature] = fr_merged_stage2_step2[player_stats_feature].fillna(0)
data_analysis = fr_merged_stage2_step2.copy()
print("The dataset has been cleaned completely!")

The dataset has been cleaned completely!


In [78]:
final_d1 = data_analysis.copy()
test_d1 = fr_merged_stage2_step2.copy()
print(f"Shape (master dataframe): {data_analysis.shape}")
print(f"Shape (cleaned dataframe): {test_d1.shape}")

Shape (master dataframe): (440, 396)
Shape (cleaned dataframe): (440, 396)


## VII. Feature Engineering, Feature Reduction and Feature Selection for the Final Dataset

In [79]:
teams_stats = [
    'HT_strength', 'HT_strength_overall_home', 'HT_strength_attack_home', 'HT_strength_defence_home',
    'HT_AvgGF_L5', 'HT_AvgGA_L5', 'HG_HT_AvgGF_L5', 'HG_HT_AvgGA_L5', 'HT_AvgShots_L5',
    'HT_ShotAccuracy_L5', 'HT_ShotConversion_L5', 'HT_CS_L5', 'HT_WinRate_L5'
]
players_stats = [
    col.replace('HT_', '').replace('AT_', '')
    for col in test_d1.columns 
    if col.startswith('HT_GK') or col.startswith('HT_DEF') or col.startswith('HT_MID') or col.startswith('HT_FWD')
]
elo_features = ['ht_match_elo', 'at_match_elo']
match_features = ['home_possession', 'away_possession', 'home_expected_goals_xg', 'away_expected_goals_xg',
                  'home_big_chances', 'away_big_chances', 'home_big_chances_missed', 'away_big_chances_missed',
                  'home_xg_open_play', 'away_xg_open_play', 'home_xg_set_play', 'away_xg_set_play', 
                  'home_non_penalty_xg', 'away_non_penalty_xg', 'home_xg_on_target_xgot', 'away_xg_on_target_xgot',
                  'home_touches_in_opposition_box', 'away_touches_in_opposition_box']

In [80]:
player_base_features = [col.replace('HT_', '') for col in test_d1.columns 
                        if col.startswith('HT_GK') or col.startswith('HT_DEF') or col.startswith('HT_MID') or col.startswith('HT_FWD')]
player_base_features = sorted(list(set(player_base_features)))

original_cols_to_drop = []

for home_col in teams_stats:
    base_col = home_col.replace('HT_', '')
    away_col = f'AT_{base_col}'
    if away_col in test_d1.columns:
        test_d1[f'{base_col}_Diff'] = test_d1[home_col] - test_d1[away_col]
        original_cols_to_drop.extend([home_col, away_col])

for features in player_base_features:
    home = f'HT_{features}'
    away = f'AT_{features}'
    test_d1[f'{features}_Diff'] = test_d1[home] - test_d1[away]
    original_cols_to_drop.extend([home, away])

original_cols_to_drop.extend(elo_features)

for idx in range(0, len(match_features), 2):
    home_col = match_features[idx]
    away_col = match_features[idx+1]
    new_col = f'{home_col.replace('home_', '').replace('_xg', '')}_Diff'
    test_d1[new_col] = test_d1[home_col] - test_d1[away_col]
    original_cols_to_drop.extend([home_col, away_col])

original_cols_to_drop = list(set(original_cols_to_drop))

future_cols_to_drop_set = set(future_cols_to_drop)
original_cols_to_drop_set = set(original_cols_to_drop)
union_cols_to_drop_set = future_cols_to_drop_set.union(original_cols_to_drop_set)
union_cols_to_drop = list(union_cols_to_drop_set)

data_analysis = test_d1.copy()
test_d2 = test_d1.drop(columns=union_cols_to_drop, errors='ignore')

print("Feature Engineering and Feature Reduction done successfully!")

Feature Engineering and Feature Reduction done successfully!


C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_26420\449208135.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_d1[new_col] = test_d1[home_col] - test_d1[away_col]


In [81]:
print(f"Shape of master dataframe (after): {data_analysis.shape}")
print(f"Shape of cleaned dataframe (after): {test_d2.shape}")

Shape of master dataframe (after): (440, 491)
Shape of cleaned dataframe (after): (440, 178)


## VIII. Data Preparation for Model Training

#### For Classification: Target Feature = 'FTR' (Full Time Result) - This must be encoded into numerical values, using Label Encoding
#### Along with it, 'HTR' (Half Time Result) should also be encoded in a similar fashion since it poses high predictive strength

In [82]:
df = test_d2.copy()

In [83]:
print(f"Final Shape of the Dataset (before): {df.shape}")

Final Shape of the Dataset (before): (440, 178)


In [84]:
df_removal = df.copy()
output_target = df_removal['FTR']
output_regression_home = df_removal['FTHG']
output_regression_away = df_removal['FTAG']
dropped_cols = [
    'FTHG', 'FTAG',
    'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR',
    'Date', 'season', 'match_id', 'MatchUp',
    'HomeTeam', 'AwayTeam', 'HT_Last_Win', 'AT_Last_Win',
    'HT_code', 'AT_code', 'HT_Points', 'AT_Points',
    'H2H_HT_Points_L5', 'H2H_AT_Points_L5',
    'HT_elo', 'AT_elo',
    'HT_strength_overall_home', 'AT_strength_overall_away',
    'HT_strength_attack_home', 'AT_strength_attack_away',
    'HT_strength_defence_home', 'AT_strength_defence_away',
    'HG_HT_AvgGF_L5', 'HG_HT_AvgGA_L5', 'HG_HT_AvgGD_L5', 'AG_AT_AvgGF_L5', 'AG_AT_AvgGA_L5', 'AG_AT_AvgGD_L5',
    'HT_Encoded_Strength', 'AT_Encoded_Strength', 
    'GK_L5_Avg_team_goals_conceded_Diff' 
]
df_removal = df_removal.drop(columns=dropped_cols, errors='ignore')
print(f"Final Shape of the dataset (after): {df_removal.shape}")

Final Shape of the dataset (after): (440, 138)


In [85]:
X_c1 = df_removal.drop(columns='FTR', errors='ignore').copy()
y_c1 = pd.get_dummies(output_target, prefix='Result')
print("Data successfully split into Input and Output features")

Data successfully split into Input and Output features


## IX. Train-Test Splitting & Model Training 

### Train & Test: Chronological Split

In [86]:
TEST_SIZE_RATIO = 0.25
split_point = int(len(X_c1) * (1 - TEST_SIZE_RATIO))

X_train_c1 = X_c1.iloc[:split_point].copy()
X_test_c1 = X_c1.iloc[split_point:].copy()

y_train_c1 = y_c1.iloc[:split_point].copy()
y_test_c1 = y_c1.iloc[split_point:].copy()

y_train_r_home = output_regression_home.iloc[:split_point].copy()
y_test_r_home = output_regression_home.iloc[split_point:].copy()

y_train_r_away = output_regression_away.iloc[:split_point].copy()
y_test_r_away = output_regression_away.iloc[split_point:].copy()

print("Classification data successfully split chronologically into training & testing sets")
print(f"Shape of Training Set: {X_train_c1.shape}")
print(f"Shape of Test Set: {X_test_c1.shape}")

Classification data successfully split chronologically into training & testing sets
Shape of Training Set: (330, 137)
Shape of Test Set: (110, 137)


### Scaling the Non-Target numerical features 

In [87]:
scaler = StandardScaler()
X_train_c1[X_train_c1.columns] = scaler.fit_transform(X_train_c1[X_train_c1.columns])
X_test_c1[X_test_c1.columns] = scaler.transform(X_test_c1[X_test_c1.columns])
print("All features have been scaled correctly (fit on train, transform on test)")

All features have been scaled correctly (fit on train, transform on test)


### Renaming the columns for model training

In [88]:
X_train_c1.columns = X_train_c1.columns.str.replace('>', '_GT_', regex=False).str.replace('<', '_LT_', regex=False).str.replace('.', '_', regex=False)
X_test_c1.columns = X_test_c1.columns.str.replace('>', '_GT_', regex=False).str.replace('<', '_LT_', regex=False).str.replace('.', '_', regex=False)

### Label Encoding: For encoding output feature, Establishing class weight

In [89]:
y_train_labels = y_train_c1.idxmax(axis=1)
y_test_labels = y_test_c1.idxmax(axis=1)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(pd.concat([y_train_labels, y_test_labels]).unique()) 
y_train_encoded = le.transform(y_train_labels)
y_test_encoded = le.transform(y_test_labels)

In [90]:
classes = np.unique(y_train_encoded)
class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=classes, 
    y=y_train_encoded 
)
sample_weight = np.array([class_weights[i] for i in y_train_encoded])

In [91]:
X_train_c1.shape

(330, 137)

In [92]:
X_test_c1.shape

(110, 137)

### Classification Model

In [97]:
print("Initializing the XGBoost Ensemble Model...")
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    n_estimators=1000,
    learning_rate=0.09,
    num_class=3, 
    eval_metric='mlogloss',
    random_state=42
)
print("Training the model now...")
xgb_model.fit(X_train_c1, y_train_encoded, sample_weight=sample_weight)
print("Predicting on test data...")
prediction = xgb_model.predict(X_test_c1)
print("Evaluating the performance of the model...")
accuracy = accuracy_score(y_test_encoded, prediction) 
print(f"ACCURACY (Fine-Tuned): {accuracy * 100:.4f}%")
report = classification_report(y_test_encoded, prediction)
print("CLASSIFICATION REPORT (Fine-Tuned):")
print(report)

Initializing the XGBoost Ensemble Model...
Training the model now...
Predicting on test data...
Evaluating the performance of the model...
ACCURACY (Fine-Tuned): 71.8182%
CLASSIFICATION REPORT (Fine-Tuned):
              precision    recall  f1-score   support

           0       0.66      0.82      0.73        33
           1       0.59      0.36      0.44        28
           2       0.81      0.86      0.83        49

    accuracy                           0.72       110
   macro avg       0.68      0.68      0.67       110
weighted avg       0.71      0.72      0.70       110



### Feature Importance Analysis

In [98]:
importance_df = pd.DataFrame({
    'Feature': X_train_c1.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values(by='Importance', ascending=False)
imp_features = importance_df['Feature'].to_list()
X_train_1 = X_train_c1[imp_features].copy()
X_test_1 = X_test_c1[imp_features].copy()

### Stratified K-Fold and Grid Search Cross Validation

In [88]:
# param_grid = {
#     'max_depth': [3, 4, 5], 
#     'learning_rate': [0.01, 0.05, 0.09], 
#     'min_child_weight': [0.5, 1, 2], 
#     'gamma': [0.5, 1], 
# }

# xgb_clf = xgb.XGBClassifier(
#     objective='multi:softmax',
#     n_estimators=1000, 
#     num_class=3,
#     eval_metric='mlogloss',
#     random_state=42
# )

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# grid_search = GridSearchCV(
#     estimator=xgb_clf,
#     param_grid=param_grid,
#     scoring='f1_macro', 
#     cv=skf,
#     verbose=1,
#     n_jobs=-1 
# )

# print("Starting Grid Search on top 60 features...")
# grid_search.fit(X_train_1, y_train_encoded, sample_weight=sw)

# best_model = grid_search.best_estimator_
# print("\nBest Parameters found:", grid_search.best_params_)
# print("Best Cross-Validation Macro F1 Score:", grid_search.best_score_)

In [99]:
best_params = {'gamma': 1, 'learning_rate': 0.09, 'max_depth': 4, 'min_child_weight': 2}
final_xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    n_estimators=1000,
    num_class=3,
    eval_metric='mlogloss',
    random_state=42, 
    **best_params
)
print("Training the model now...")
final_xgb_model.fit(X_train_1, y_train_encoded, sample_weight=sample_weight) 
print("Predicting on test data...")
best_prediction = final_xgb_model.predict(X_test_1)
best_accuracy = accuracy_score(y_test_encoded, best_prediction)
best_report = classification_report(y_test_encoded, best_prediction)
print(f"\nBEST ACCURACY (Optimized Model): {best_accuracy * 100:.4f}%")
print("BEST CLASSIFICATION REPORT (Optimized Model):")
print(best_report)

Training the model now...
Predicting on test data...

BEST ACCURACY (Optimized Model): 67.2727%
BEST CLASSIFICATION REPORT (Optimized Model):
              precision    recall  f1-score   support

           0       0.68      0.70      0.69        33
           1       0.43      0.43      0.43        28
           2       0.81      0.80      0.80        49

    accuracy                           0.67       110
   macro avg       0.64      0.64      0.64       110
weighted avg       0.67      0.67      0.67       110



### Regression Model

In [94]:
rf_home_goals = RandomForestRegressor(
    n_estimators=1000,     
    max_depth=10,         
    random_state=42,
    n_jobs=-1             
)
rf_away_goals = RandomForestRegressor(
    n_estimators=1000,     
    max_depth=10,         
    random_state=42,
    n_jobs=-1             
)

print("Starting training for Home Goals Regressor...")
rf_home_goals.fit(X_train_c1, y_train_r_home)
print("Starting training for Away Goals Regressor...")
rf_away_goals.fit(X_train_c1, y_train_r_away)

pred_home_goals_float = rf_home_goals.predict(X_test_c1)
pred_away_goals_float = rf_away_goals.predict(X_test_c1)

pred_home_goals_int = np.round(np.maximum(0, pred_home_goals_float)).astype(int)
pred_away_goals_int = np.round(np.maximum(0, pred_away_goals_float)).astype(int)

mae_home = mean_absolute_error(y_test_r_home, pred_home_goals_int)
mae_away = mean_absolute_error(y_test_r_away, pred_away_goals_int)

mse_home = mean_squared_error(y_test_r_home, pred_home_goals_int)
mse_away = mean_squared_error(y_test_r_away, pred_away_goals_int)

print("\nREGRESSION MODEL PERFORMANCE (Score Prediction)")

# Display Home Goal Metrics
print("Home Goals (FTHG) Metrics:")
print(f"Mean Absolute Error (MAE): {mae_home:.4f}")
print(f"Mean Squared Error (MSE):  {mse_home:.4f}")

# Display Away Goal Metrics
print("\nAway Goals (FTAG) Metrics:")
print(f"Mean Absolute Error (MAE): {mae_away:.4f}")
print(f"Mean Squared Error (MSE):  {mse_away:.4f}")

# Overall Model Health
print("\nOverall Scoreline Health")
print(f"Average MAE (Scoreline): {(mae_home + mae_away) / 2:.4f}")

Starting training for Home Goals Regressor...
Starting training for Away Goals Regressor...

REGRESSION MODEL PERFORMANCE (Score Prediction)
Home Goals (FTHG) Metrics:
Mean Absolute Error (MAE): 0.7545
Mean Squared Error (MSE):  0.9909

Away Goals (FTAG) Metrics:
Mean Absolute Error (MAE): 0.7545
Mean Squared Error (MSE):  0.9545

Overall Scoreline Health
Average MAE (Scoreline): 0.7545


In [95]:
for idx in range(10, 20):
    print(f"Actual Score: {y_test_r_home.iloc[idx]} - {y_test_r_away.iloc[idx]} | Predicted Score: {pred_home_goals_int[idx]} - {pred_away_goals_int[idx]}")

Actual Score: 1.0 - 0.0 | Predicted Score: 2 - 1
Actual Score: 1.0 - 0.0 | Predicted Score: 2 - 1
Actual Score: 2.0 - 2.0 | Predicted Score: 1 - 1
Actual Score: 2.0 - 0.0 | Predicted Score: 2 - 1
Actual Score: 1.0 - 2.0 | Predicted Score: 1 - 1
Actual Score: 4.0 - 3.0 | Predicted Score: 2 - 1
Actual Score: 1.0 - 1.0 | Predicted Score: 1 - 2
Actual Score: 1.0 - 1.0 | Predicted Score: 1 - 1
Actual Score: 3.0 - 1.0 | Predicted Score: 2 - 1
Actual Score: 1.0 - 1.0 | Predicted Score: 1 - 1


In [86]:
import joblib

In [ ]:
df = joblib.load(r"C:\PROJECT\data_artifacts\master_data.pkl")

In [100]:
df = df.sort_values(by='Date', ascending=False).reset_index(drop=True)

In [104]:
df[(df['HomeTeam'] == 'Liverpool') & (df['AwayTeam'] == 'Aston Villa')]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,MID_L5_Avg_xg_Diff,possession_Diff,expected_goals_Diff,big_chances_Diff,big_chances_missed_Diff,xg_open_play_Diff,xg_set_play_Diff,non_penalty_Diff,xg_on_targetot_Diff,touches_in_opposition_box_Diff
335,2024-11-09,Liverpool,Aston Villa,2.0,0.0,H,1.0,0.0,H,D Coote,...,0.0299,24.0,0.64,-1.0,-3.0,1.34,-0.69,0.64,0.64,2.0


In [89]:
test = df[(df['HomeTeam'] == 'Liverpool') & (df['AwayTeam'] == 'Brentford')]

In [96]:
ff = joblib.load(r"C:\PROJECT\data_artifacts\final_features.pkl")